In [1]:
import torch
import torchvision
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import torchvision.transforms.functional as fn
from torch import nn
import torch.nn.functional as F
import wandb
import torch.optim as optim
from tqdm import tqdm
import sys
import autoreload
%load_ext autoreload
%autoreload 2
sys.path.append('../scripts/')
from save_model_every_epoch_torch import save_model
import warnings
warnings.filterwarnings("ignore")
from metrics import find_metrics, find_metrics_macro

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
multiple_gpus = True
if torch.cuda.is_available():
    if torch.cuda.device_count() > 1:
        multiple_gpus = True
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
csv_file = pd.read_csv('../../files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../files/train_images/' + x)

In [4]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size)
    train, val = train_test_split(train, test_size=test_size)
    return train, val, test

In [5]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [6]:
encoder = encoding_data(csv_file)

In [7]:
class DolphinDataset(Dataset):
    def __init__(self, csv_file, encoder):
        self.csv_file = csv_file
        self.images = csv_file['updated_paths'].values.tolist()
        self.labels = csv_file['species'].values.tolist()
        self.encoder = encoder
        self.encoded_labels = self.encoder.transform(np.array(self.labels).reshape(-1, 1)).toarray()
        self.convert_rgb = torchvision.transforms.Lambda(lambda x: x.repeat(3, 1, 1))

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.images[idx])
        img = torchvision.io.decode_jpeg(img)
        if img.shape[0] != 3:
            img = self.convert_rgb(img)
        img = torchvision.transforms.functional.resize(img, (512, 512))
        img = img / 255.0
        return img, torch.Tensor(self.encoded_labels[idx])

In [8]:
train, val, test = split_datasets(csv_file, test_size = 0.01)

In [9]:
# incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)

In [10]:
def get_model():
    incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)
    incep_model.fc = nn.Linear(2048, 30)
    return incep_model

In [11]:
categorical_cross_entropy = nn.CrossEntropyLoss()

In [12]:
def train_model(train_dataset, val_dataset, epochs, batch_size, load_weights=None):
    model = get_model()
    wandb.init(project='dolphin',
               config = {
                   'arch' : 'InceptionV4'
               })
    data_pointers = {
        'train' : train_dataset,
        'val' : val_dataset,
    }
    if load_weights != None:
        model.load_state_dict(torch.load(load_weights)['model_state_dict'])
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    if next(model.parameters()).is_cuda == False:
        if multiple_gpus == True:
            model = nn.DataParallel(model)
        model = model.to(device)
    for epoch in range(epochs):
        train_loss= 0.0
        train_prec, train_rec = 0.0, 0.0
        val_loss = 0.0
        val_prec, val_rec = 0.0, 0.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss, running_precision, running_recall = 0.0, 0.0, 0.0
            with tqdm(data_pointers[phase], unit='batch') as tepoch:
                for imgs, labels in tepoch:
                    tepoch.set_description(f'Epoch: {epoch}')
                    imgs = imgs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            outputs = model(imgs)[0]
                        else:
                            outputs = model(imgs)
                        loss = categorical_cross_entropy(outputs, labels)
                        prec, rec = find_metrics_macro(outputs, labels, batch_size=batch_size)
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                    running_loss += loss.item()
                    running_precision += prec.item()
                    running_recall += rec.item()
                    tepoch.set_postfix(loss = loss.item(), precision = prec.item(), recall = rec.item())
            if phase == 'train':
                train_loss = running_loss / len(train_dataset)
                train_prec = running_precision / len(train_dataset)
                train_rec = running_recall / len(train_dataset)
                print(f'{phase} Loss: {float(train_loss)}')
                print(f'{phase} Precision: {float(train_prec)}')
                print(f'{phase} Recall: {float(train_rec)}')
            else:
                val_loss = running_loss / len(val_dataset)
                val_prec = running_precision / len(val_dataset)
                val_rec = running_recall / len(val_dataset)
                print(f'{phase} Loss: {float(val_loss)}')
                print(f'{phase} Precision: {float(val_prec)}')
                print(f'{phase} Recall: {float(val_rec)}')
        wandb.log({
            'train_loss' : train_loss,
            'val_loss' : val_loss,
            'train_precision' : train_prec,
            'train_recall' : train_rec,
            'val_precision' : val_prec,
            'val_recall' : val_rec
        })

In [13]:
batch_size = 228
train_dataset = DolphinDataset(train, encoder)
val_dataset = DolphinDataset(val, encoder)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 24, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, epochs=10)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch: 0:  77%|███████▋  | 169/220 [03:54<01:02,  1.23s/batch, loss=2.84, precision=0.0415, recall=0.0187] 

In [18]:
preds = np.random.random((32, 30))
random_g_truth = np.random.randint(0, 30, 32)
g_truth = np.zeros((32, 30))
g_truth[np.arange(32), random_g_truth] = 1

In [20]:
g_truth = torch.Tensor(g_truth)
preds = torch.Tensor(preds)

In [26]:
x, y, z = find_metrics_macro(preds, g_truth)

In [37]:
temp = torch.sum(torch.nan_to_num(x / (x + y))) / 30

In [38]:
temp

tensor(0.0167)

In [32]:
x[0] / torch.Tensor([0])

tensor([nan])

In [30]:
y

tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 0, 1, 1, 1, 1, 1, 3, 0, 1, 1, 0, 5, 1, 2,
        0, 0, 3, 1, 1, 2])